In [0]:
from pyspark.sql.functions import sum as spark_sum

# Load cleaned Silver table
df_cleaned = spark.table("silver_healthcare_orders")

# ==========================
# Revenue by product category (Gold layer)
# ==========================
df_category_sales = (
    df_cleaned.groupBy("service_category")
              .agg(spark_sum("total_value").alias("total_revenue"))
)

# Save as Gold table
df_category_sales.createOrReplaceTempView("tmp_service_category_sales")

spark.sql("""
    CREATE OR REPLACE TABLE gold_healthcare_service_category_sales AS
    SELECT * FROM tmp_service_category_sales
""")

# ==========================
# Daily revenue (Gold layer)
# ==========================
df_daily_sales = (
    df_cleaned.groupBy("order_date")
              .agg(spark_sum("total_value").alias("daily_revenue"))
)

# Save as Gold table
df_daily_sales.createOrReplaceTempView("tmp_healthcare_daily_sales")

spark.sql("""
    CREATE OR REPLACE TABLE gold_healthcare_daily_sales AS
    SELECT * FROM tmp_healthcare_daily_sales
""")
